## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [create task with image_name](#create-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API_KEY')
swan_orchestrator = swan.resource(api_key=api_key, service_name='Orchestrator')

### Show available hardware information

In [2]:
hardwares_info = swan_orchestrator.get_hardware_config()
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'region': ['North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'region': ['North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'region': ['North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 73,
  'name': 'R1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 32 GiB',
  'type': 'GPU',
  'region': ['North Carolina-US'],
  'price': '22.0',
  'status': 'available'}]

### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [3]:
result = swan_orchestrator.create_task(
    image_name="hello-world",
    wallet_address=wallet_address,
    private_key=private_key
)
print(json.dumps(result, indent=2))
task_uuid = result['id']

{
  "data": {
    "config_id": 1,
    "created_at": 1719612207,
    "duration": 3600,
    "ended_at": null,
    "error_code": null,
    "id": 2459,
    "order_type": "Creation",
    "refund_tx_hash": null,
    "region": "global",
    "space_id": null,
    "start_in": 300,
    "started_at": 1719612207,
    "status": "pending_payment_confirm",
    "task_uuid": "a5f9278e-99a6-401b-bf54-fb86b36b0fd4",
    "tx_hash": "0xfccf187a0e384034d332b2fd2bcd3d26a9ba6b7c89ed5100b245c0ef6bed0cd1",
    "updated_at": 1719612217,
    "uuid": "4c711126-a3f1-4943-8024-3b0312ed5359"
  },
  "message": "Query order status success.",
  "status": "success",
  "tx_hash": "0xfccf187a0e384034d332b2fd2bcd3d26a9ba6b7c89ed5100b245c0ef6bed0cd1",
  "id": "a5f9278e-99a6-401b-bf54-fb86b36b0fd4"
}


### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [4]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "beneficiary": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "cp_account_address": "0x4D81F4277667862046802bDdcdB667704bEFbbAB",
        "created_at": 1718385670,
        "id": 66,
        "lat": 45.5075,
        "lon": -73.5887,
        "multi_address": [
          "/ip4/38.140.46.60/tcp/8086"
        ],
        "node_id": "0478a886cb6e191a0531fb6f8a7da4c0fbf0ec220af6d49e622f7e2da514f0ea205ed77918796c66ae642f0996dbf0ecc4bb7f1dd949705d1306abd1c14b60fb8a",
        "online": 1,
        "owner_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "region": "Quebec-CA",
        "task_types": "[1, 2, 3]",
        "updated_at": 1719612229,
        "version": "2.0",
        "worker_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373"
      },
      {
        "beneficiary": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "cp_account_address": "0xd63853b23Af4cc3b37377368dd33F09b8567aCea",
        "created_at": 

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [5]:
result_url = swan_orchestrator.get_real_url(task_uuid)
print(result_url)

['https://vxfxfs2vcr.dev2.crosschain.computer', 'https://at0xdkllnm.cp.filezoo.com.cn']


Check the response codes of the result_url

In [6]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

for url in result_url:
    response = requests.get(url, headers=headers)

    try:
        print(json.dumps(response.json(), indent=4))
    except Exception as e:
        print(e)
        print(response)


Expecting value: line 1 column 1 (char 0)
<Response [200]>
Expecting value: line 1 column 1 (char 0)
<Response [503]>
